<a href="https://colab.research.google.com/github/vighc/SupplyChainData/blob/main/SupplyChainData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install require packages
!pip install xlsxwriter
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 2.6 MB/s eta 0:00:00


In [2]:
# Download [Pan Pacific -Input Data.xlsx] from Git
!wget https://raw.githubusercontent.com/vighc/SupplyChainData/main/Pan%20Pacific%20-Input%20Data.xlsx

--2025-04-02 07:26:26--  https://raw.githubusercontent.com/vighc/SupplyChainData/main/Pan%20Pacific%20-Input%20Data.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16086 (16K) [application/octet-stream]
Saving to: ‘Pan Pacific -Input Data.xlsx’

Pan Pacific -Input  100%[===================>]  15.71K  --.-KB/s    in 0.001s  

2025-04-02 07:26:26 (11.3 MB/s) - ‘Pan Pacific -Input Data.xlsx’ saved [16086/16086]



In [3]:
import pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Initialize Spark Session
spark = SparkSession.builder.appName("ReadExcel").getOrCreate()

# Read Excel File
pdf = ps.read_excel("/content/Pan Pacific -Input Data.xlsx")
df = spark.createDataFrame(pdf)

# Replace NaN Values
df = df.replace("NaN", None)
df.show()

+-----+--------------------+--------------------+----------------+---------------+-----+----------+-----------+-----------+-------+
| Code|                Name|            Address1|        Address2|           City|State|       Zip|Salesperson|Home Branch|   Type|
+-----+--------------------+--------------------+----------------+---------------+-----+----------+-----------+-----------+-------+
|32604|PAN-PACIFIC PLUMB...|     18250 EUCLID ST|            NULL|FOUNTAIN VALLEY|   CA|92708-6175|      STANR|       LASF|Bill To|
|43861|PAN PACIFIC / EUC...| 18250 EUCLID STREET|            NULL|FOUNTAIN VALLEY|   CA|92708-6112|      STANR|       LASF|Ship To|
|59835|PAN PACIFIC / FAB...|    18050 NEWHOPE ST|            NULL|FOUNTAIN VALLEY|   CA|92708-6130|      STANR|       LASF|Ship To|
|60064|PAN-PACIFIC / WIL...|    1001 YOSEMITE DR|            NULL|       MILPITAS|   CA|95035-5409|    CCASTRO|        MIL|Ship To|
|60085|PAN-PACIFIC / KAI...|945 BALDWIN PARK ...|            NULL|   BALDWIN

In [4]:
# Create Customer DataFrame
customer_df = df.filter(F.col("Type")=="Bill To").select(
    F.lit("12345").alias("id"),
    F.col("Name").alias("name"),
    F.col("Code").alias("account_number"),
    F.lit("{xyz.com}").alias("domain_name"),
    F.lit("xyz").alias("tenant_id")
)
customer_df.show()

+-----+--------------------+--------------+-----------+---------+
|   id|                name|account_number|domain_name|tenant_id|
+-----+--------------------+--------------+-----------+---------+
|12345|PAN-PACIFIC PLUMB...|         32604|  {xyz.com}|      xyz|
+-----+--------------------+--------------+-----------+---------+



In [5]:
# Create Invoice DataFrame
invoice_df = df.join(customer_df.drop('name'), F.col('Code') == F.col('account_number'), 'inner').withColumn("Type", F.regexp_replace(F.col("Type"), "Bill", "Ship"))

# Create Ship To DataFrame
shipto_df = df.join(customer_df.drop('name'), F.col('Type') == F.lit('Ship To'), 'inner')

# Create Bill To DataFrame
billto_df = df.join(customer_df.drop('name'), F.col('Code') == F.col('account_number'), 'inner').withColumn("Code", F.lit(None))

# Combine DataFrames
outputData_Union_df = invoice_df.union(shipto_df)
outputData_Union_df = outputData_Union_df.union(billto_df)
outputData_Union_df.show()

+-----+--------------------+--------------------+----------------+---------------+-----+----------+-----------+-----------+-------+-----+--------------+-----------+---------+
| Code|                Name|            Address1|        Address2|           City|State|       Zip|Salesperson|Home Branch|   Type|   id|account_number|domain_name|tenant_id|
+-----+--------------------+--------------------+----------------+---------------+-----+----------+-----------+-----------+-------+-----+--------------+-----------+---------+
|32604|PAN-PACIFIC PLUMB...|     18250 EUCLID ST|            NULL|FOUNTAIN VALLEY|   CA|92708-6175|      STANR|       LASF|Ship To|12345|         32604|  {xyz.com}|      xyz|
|43861|PAN PACIFIC / EUC...| 18250 EUCLID STREET|            NULL|FOUNTAIN VALLEY|   CA|92708-6112|      STANR|       LASF|Ship To|12345|         32604|  {xyz.com}|      xyz|
|59835|PAN PACIFIC / FAB...|    18050 NEWHOPE ST|            NULL|FOUNTAIN VALLEY|   CA|92708-6130|      STANR|       LASF|Sh

In [6]:
# Mapping to Standard Layout
outputData_df = outputData_Union_df.withColumn('InputId', F.row_number().over(Window.orderBy(F.col("Code").asc_nulls_last()))) \
    .withColumn('country', F.when(F.col('Zip').contains('-'), "US").otherwise("USA")) \
    .withColumn('type', F.lower(F.regexp_replace(F.concat(F.lit("customer_"), F.col("Type")), ' ', '')))

outputData_df = outputData_df.select(
    F.col("InputId").alias("id"),
    F.col("Name").alias("name"),
    F.col("Address1").alias("address_1"),
    F.col("Address2").alias("address_2"),
    F.lit(None).alias("address_3"),
    F.col("City").alias("city"),
    F.col("State").alias("state"),
    "country",
    F.col("Zip").alias("zip"),
    F.col("tenant_id").alias("tenant_id"),
    F.lit("FALSE").alias("deleted"),
    F.col("Type").alias("type"),
    F.col("Code").alias("shipto_no"),
    F.col("id").alias("customer_id"),
)
outputData_df.show()

+---+--------------------+--------------------+----------------+---------+---------------+-----+-------+----------+---------+-------+---------------+---------+-----------+
| id|                name|           address_1|       address_2|address_3|           city|state|country|       zip|tenant_id|deleted|           type|shipto_no|customer_id|
+---+--------------------+--------------------+----------------+---------+---------------+-----+-------+----------+---------+-------+---------------+---------+-----------+
|  1|PAN-PACIFIC PLUMB...|     18250 EUCLID ST|            NULL|     NULL|FOUNTAIN VALLEY|   CA|     US|92708-6175|      xyz|  FALSE|customer_shipto|    32604|      12345|
|  2|PAN PACIFIC / EUC...| 18250 EUCLID STREET|            NULL|     NULL|FOUNTAIN VALLEY|   CA|     US|92708-6112|      xyz|  FALSE|customer_shipto|    43861|      12345|
|  3|PAN PACIFIC / FAB...|    18050 NEWHOPE ST|            NULL|     NULL|FOUNTAIN VALLEY|   CA|     US|92708-6130|      xyz|  FALSE|custome

In [7]:
# Convert to Pandas DataFrames
outputData_pd = outputData_df.toPandas()
customer_pd = customer_df.toPandas()

# Write to Excel
excel_path = "panpacific_outputData.xlsx"
with ps.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    outputData_pd.to_excel(writer, sheet_name="outputData", index=False)
    customer_pd.to_excel(writer, sheet_name="customer", index=False)